In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from datetime import datetime, date
from pyspark.sql import Row

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/10 09:42:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

In [10]:
import pandas as pd
sample = pd.read_csv("play_by_play_sample.csv")
sample.head(100)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,29600012,0,12,0,1,14:43 PM,12:00,NaN,Start of 1st Period (14:43 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,29600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,NaN,NaN,...,Suns,PHX,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0
2,29600012,3,2,1,1,14:51 PM,11:45,NaN,NaN,MISS Cassell 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
3,29600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,29600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29600012,101,3,12,1,15:12 PM,2:05,MISS Campbell Free Throw 2 of 2,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
96,29600012,102,4,0,1,15:12 PM,2:05,NaN,NaN,Finley REBOUND (Off:0 Def:2),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
97,29600012,103,1,1,1,15:13 PM,1:50,NaN,NaN,Cassell 25' 3PT Jump Shot (7 PTS) (Horry 2 AST),...,Suns,PHX,0.0,0,NaN,NaN,NaN,NaN,NaN,0
98,29600012,104,5,1,1,15:13 PM,1:26,Rooks Bad Pass Turnover (P1.T4),NaN,Horry STEAL (2 STL),...,Suns,PHX,0.0,0,NaN,NaN,NaN,NaN,NaN,0


In [11]:
def time_to_seconds(time_str):
    mins, secs = map(int, time_str.split(":"))
    return mins * 60 + secs


sample["pctimestring"] = sample["pctimestring"].apply(time_to_seconds)
last_5_mins_df = sample[(sample["period"] == 4) & (sample["pctimestring"] <= 300)]
last_5_mins_df

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
416,29600012,437,2,1,4,16:58 PM,295,NaN,NaN,MISS Manning 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
417,29600012,438,4,0,4,16:58 PM,293,Campbell REBOUND (Off:1 Def:6),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
418,29600012,439,2,1,4,16:58 PM,279,MISS Van Exel 24' 3PT Jump Shot,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
419,29600012,440,4,0,4,16:58 PM,277,NaN,NaN,Tisdale REBOUND (Off:2 Def:2),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
420,29600012,441,6,1,4,16:59 PM,257,Campbell P.FOUL (P3.T2),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9867,29600022,476,4,0,4,13:17 PM,14,Person REBOUND (Off:1 Def:9),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
9868,29600022,477,2,1,4,13:17 PM,6,MISS Green 3PT Jump Shot,NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
9869,29600022,479,4,0,4,13:17 PM,4,Person REBOUND (Off:2 Def:9),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
9870,29600022,478,1,1,4,13:17 PM,4,Person 9' Jump Shot (13 PTS),NaN,NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


In [15]:
# perform same operations with spark
df = spark.read.csv("play_by_play_sample.csv")
df.filter("\"pctimestring\" == 4")

24/08/10 09:44:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `pctimestring` cannot be resolved. Did you mean one of the following? [`_c0`, `_c1`, `_c10`, `_c11`, `_c12`].; line 1 pos 0;
'Filter ('pctimestring = 4)
+- Relation [_c0#102,_c1#103,_c2#104,_c3#105,_c4#106,_c5#107,_c6#108,_c7#109,_c8#110,_c9#111,_c10#112,_c11#113,_c12#114,_c13#115,_c14#116,_c15#117,_c16#118,_c17#119,_c18#120,_c19#121,_c20#122,_c21#123,_c22#124,_c23#125,... 10 more fields] csv
